In [2]:
import faiss
import numpy as np

USE_GPU = False

dataset_name = 'redpajama_20b'



if "c4_news" in dataset_name:
    # Load your existing flat index
    flat_index = faiss.read_index(f'/home/aiops/zhuty/ret_pretraining_data/id_added/{dataset_name}/dense_index/shard_full/flatindex')
else:
    # Load your existing flat index
    flat_index = faiss.read_index(f'/home/aiops/zhuty/ret_pretraining_data/id_added/{dataset_name}/dense_index/shard_full/index')

In [3]:
from typing import List

def load_docids(docid_path: str) -> List[str]:
    id_f = open(docid_path, 'r')
    docids = [line.rstrip() for line in id_f.readlines()]
    id_f.close()
    return docids
docids = load_docids(f'/home/aiops/zhuty/ret_pretraining_data/id_added/{dataset_name}/dense_index/shard_full/docid')

In [4]:
flat_index.ntotal

8900000

In [5]:
embedding = f"/home/aiops/zhuty/ret_pretraining_data/id_added/{dataset_name}/dense_queries/chunk_0/embedding.pkl"
embedding = np.load(embedding, allow_pickle=True)


In [6]:
embedding['embedding'][0].shape

(768,)

In [7]:
embedding['text'][3]

'Basic number formating is supported in both SVG and HTML files. Standard numbering symbol like hash (#), zero (0), comma (,) and period (.) can be used in formatting a number. Moreover, IntegraXor number formatting is very flexible, any non-digit symbol or even space can be used as decimal or separator to support localized number formating. And the group separation can be a standard 3 digits thousand separator, or 2 digits, or 4 digits or any biggger numbers separator. Below are some examples of formating and its outcome.\nEntering only one symbol will always be treated as Decimal, simply add a trailing space at the end to force it become Separator. For instance, inputting 1234567.890 into "#,### " will generate 1,234,568.'

In [ ]:
flat_index.metric_type

In [ ]:
len(embedding['embedding']  )

In [17]:
q_embeddings = np.array(embedding['embedding'].tolist())
print("q_embeddings.shape", q_embeddings.shape)
q_embeddings = q_embeddings[:512, :]
print(q_embeddings.shape)

q_embeddings.shape (100000, 768)
(100, 768)


In [18]:
import time
# time the search
start = time.time()

D, I = flat_index.search(q_embeddings, 1000)

end = time.time()
print(f"Search took {end - start} seconds")

Search took 72.73102402687073 seconds


In [32]:
embedding['embedding'][0].dot(flat_index.reconstruct(10989342))

7.0621133

In [41]:
docids.index('260_19549')

10989342

In [31]:
{key: [(docids[idx], score)
                          for score, idx in zip(distances, indexes) if idx != -1]
                    for key, distances, indexes in zip(['0_3'], D, I)}

{'0_3': [('488_26675', 13.423204),
  ('54_15364', 13.305187),
  ('464_17743', 12.685675),
  ('16_12271', 12.598089),
  ('188_10357', 12.290218),
  ('354_26802', 12.207288),
  ('107_10189', 12.097836),
  ('342_25796', 11.983659),
  ('418_5252', 11.974178),
  ('234_26553', 11.97171)]}

Draft place

In [ ]:
query_type = "first"
commands = []

ds, upper, step = 'cc', 10, 1
# ds, upper, step = 'rpwiki_en' , 1200, 50
# ds, upper, step = "redpajama_20b" , 89 , 4
# for i in range(0, 1200, step):
# for i in range(0, 500, step):
for i in range(0, upper, step):
   # if i==0:
   #      continue
# missing = ['chunk_234', 'chunk_254', 'chunk_301', 'chunk_304', 'chunk_303', 'chunk_302', 'chunk_300', 'chunk_305', 'chunk_510', 'chunk_290', 'chunk_235', 'chunk_511']
# for i in missing:
#     i = i.split("_")[1]
    commands.append((f"""sleep 5; sailctl job create bm25search{i} -g 1 --debug   --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/ ; bash range_search_bm25.sh {ds} {query_type} {i} {i+step-1}  " ;"""))
print(" ".join(commands))

In [4]:
commands = []
ds, upper, step = 'c4_news', 500, 20
# ds, upper, step = 'rpwiki_en' , 1200, 50
# ds, upper, step = "redpajama_20b" , 89 , 4
ds, upper, step = 'cc', 100, 1
query_type = "keep"
# for i in range(0, upper, step):

   # if i==0:
   #      continue

       # commands.append((f"""sleep 5; sailctl job create dsearch{i}  --debug -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh {ds} {query_type} {i} {i+step-1} ;  bash range_search_dense.sh {ds} {query_type} {i}  {i+step-1}  " ;"""))
# missing = ['chunk_238', 'chunk_410', 'chunk_309', 'chunk_137', 'chunk_136', 'chunk_356', 'chunk_0', 'chunk_392', 'chunk_114', 'chunk_31', 'chunk_82', 'chunk_426', 'chunk_389',
# 'chunk_483', 'chunk_5', 'chunk_329', 'chunk_73', 'chunk_251', 'chunk_494', 'chunk_386', 'chunk_11', 'chunk_13', 'chunk_396', 'chunk_4', 'chunk_467', 'chunk_135', 'chunk_269', 'chunk_446', 'chunk_289', 'chunk_230', 'chunk_406',
# 'chunk_93', 'chunk_16', 'chunk_62', 'chunk_1', 'chunk_330', 'chunk_511', 'chunk_470', 'chunk_332', 'chunk_297', 'chunk_346', 'chunk_115', 'chunk_316', 'chunk_326']
# for i in missing:
#     i = i.split("_")[1]
#     i = str(i)
for i in [27, 29, 33, 35, 61, 63, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 91, 93, 95, 97, 99]:
    step = 1
    commands.append((f"""sleep 5; sailctl job create {ds}dsearch{i}  --debug -g 1 -p high --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh {ds} {query_type} {i} {i+step-1} ;  bash range_search_dense.sh {ds} {query_type} {i}  {i+step-1}  " ;"""))
print(" ".join(commands))

sleep 5; sailctl job create ccdsearch27  --debug -g 1 -p high --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh cc keep 27 27 ;  bash range_search_dense.sh cc keep 27  27  " ; sleep 5; sailctl job create ccdsearch29  --debug -g 1 -p high --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh cc keep 29 29 ;  bash range_search_dense.sh cc keep 29  29  " ; sleep 5; sailctl job create ccdsearch33  --debug -g 1 -p high --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh cc keep 33 33 ;  bash range_search_dense.sh cc keep 33  33  " ; sleep 5; sailctl job create ccdsearch35  --debug -g 1 -p high --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh cc keep 35 35 ; 

In [9]:
commands = []
ds, upper, step = 'c4_news', 500, 10
# ds, upper, step = 'rpwiki_en' , 1200, 50
# ds, upper, step = "redpajama_20b" , 89 , 4
query_type = "first"
for i in range(0, upper, step):
   # if i==0:
   #      continue
# missing = ['chunk_234', 'chunk_254', 'chunk_301', 'chunk_304', 'chunk_303', 'chunk_302', 'chunk_300', 'chunk_305', 'chunk_510', 'chunk_290', 'chunk_235', 'chunk_511']
# for i in missing:
#     i = i.split("_")[1]
    if i == 0:
        continue
    commands.append(f"""sailctl job create geninf -g 1 -p low  --debug --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/inference ; bash run_batch_inf.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_120M_8k_c4_news_8k/hf_ckpt c4_news last {i} {i+step-1} ;" ;""")
print(" ".join(commands))

sailctl job create geninf -g 1 -p low  --debug --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/inference ; bash run_batch_inf.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_120M_8k_c4_news_8k/hf_ckpt c4_news last 10 19 ;" ; sailctl job create geninf -g 1 -p low  --debug --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/inference ; bash run_batch_inf.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_120M_8k_c4_news_8k/hf_ckpt c4_news last 20 29 ;" ; sailctl job create geninf -g 1 -p low  --debug --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/inference ; bash run_batch_inf.sh /home/aiops/zhuty/tinyllama/out/tiny_LLaMA_120M_8k_c4_news_8k/hf_ckpt c4_news last 30 39 ;" ; sailctl job create geninf -g 1 -p low  --debug --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/inference ; bash run_batch_inf.s

In [ ]:
sailctl job create dsearch480  --debug -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama/processing/dense_index ; bash encode_queries.sh c4_news first 510 511 ;  bash range_search_dense.sh c4_news first 510  511  " ;